^fill^README.md^here^

^fill^theory.md^here^

## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

- [spglib](https://atztogo.github.io/spglib/python-spglib.html)

In [ ]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://atztogo.github.io/spglib/python-spglib.html
import spglib

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

#### 1.2. Default calculation setup

In [ ]:
# Specify calculation style
calc_style = 'crystal_space_group'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = os.path.join(os.getcwd(), 'calculationfiles', calc_style)
    if not os.path.isdir(workingdir):
        os.mkdir(workingdir)
    os.chdir(workingdir)

# Default iprPy library directory
librarydir = os.path.join(iprPy.rootdir, '..', 'library')

### 2. Assign values for the calculation's run parameters

#### 2.1. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load_* parameters and symbols.

In [ ]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = os.path.join(librarydir, 'crystal_prototype', prototype_name+'.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

#### 2.5. Specify calculation-specific run parameters

- __symmetryprecision__ is a precision tolerance used for the atomic positions and box dimensions for determining symmetry elements.  Default value is '0.01 angstrom'.

- __primitivecell__ is a boolean flag indicating if the returned unit cell is to be primitive (True) or conventional (False).  Default value is False.

- __idealcell__ is a boolean flag indicating if the box dimensions and atomic positions are to be idealized based on the space group (True) or averaged based on their actual values (False).  Default value is True.

In [ ]:
symmetryprecision = uc.set_in_units(0.01, 'angstrom')
primitivecell = True
idealcell = True

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. crystal_space_group()

In [ ]:
^fill^calc_crystal_space_group.py(crystal_space_group)^here^

### 4. Run calculation function(s)

In [ ]:
results_dict = crystal_space_group(ucell,
                                   symprec=symmetryprecision,
                                   to_primitive=primitivecell,
                                   no_idealize=not idealcell)

### 5. Report results

#### 5.1. Display space group information

In [ ]:
for key in results_dict.keys():
    print(key)
    print(results_dict[key])
    print()